# Notebook rilut

### Bikin jelek, hapus aja

- colorCode
- total_spent, 
- customerID
- articleID
- orderDate
- productGroup (karena di-impute paksa dengan 0? atau emang ga penting?)

### Jangan dihapus

- rrp, sizeCode

### Ga pengaruh / pengaruh dikit banget

- months (labelencoder), kali dihapus, dibinerin, atau diapain
- mmdd (labelencoder), kali dihapus, dibinerin, atau diapain

kalo moths/mmdd diilangin, berkurang 3. tapi kalo months & mmdd dihilangin sekaligus, berkurang 10.

### Missing values

rrp, productGroup

### Catatan:

Logistic Regression lama banget. itu pun solvernya udah diubah. kalau masih mau pake, iternya kurangin kali?

### TODO:

- pakai PolynomialFeatures (di selain fitur harga (?) (selain price, rrp, after_voucher, unit_price, usual_unit_price, price_diff, etc))
- ekstrak fitur karena fitur adalah segalanya

In [1]:
from __future__ import division

label='returnQuantity'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from operator import itemgetter
#from sklearn.externals import joblib

def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

def feature_importances(Xdf, rf_model):
    pair=zip(Xdf.columns, rf_model.feature_importances_)
    t1=[i for i in pair]; t1
    return sorted(t1, key=itemgetter(1))

In [2]:
# Pakai make_datasets_train_only. Buat sendiri juga boleh

df=pd.read_csv('train_gue.csv')

if 'deviceID' in df.columns:
    del df['deviceID']
    
del df["colorCode"]
del df['customerID']
del df['articleID']
del df['orderDate']
#del df['productGroup']

df.head()

,orderID,sizeCode,productGroup,quantity,price,rrp,voucherAmount,paymentMethod,returnQuantity,mmdd,...,price_diff,after_voucher,order_order,choice_order,articleID_prob,colorCode_prob,sizeCode_prob,customerID_prob,ac_prob,as_prob
0,a1000001,44,3.0,1,10.0,29.984375,0.0,BPRG,0,01-01,...,0.000000,30.0,1,1,0.421143,0.521484,0.516113,0.0,0.421053,0.421053
1,a1000001,44,3.0,1,20.0,40.000000,0.0,BPRG,0,01-01,...,5.000000,30.0,2,2,0.506836,0.492432,0.516113,0.0,0.475410,0.475410
2,a1000002,38,8.0,1,35.0,50.000000,0.0,BPRG,0,01-01,...,10.000000,85.0,1,1,0.642090,0.455322,0.535645,0.5,0.616505,0.616505
3,a1000002,38,8.0,1,50.0,50.000000,0.0,BPRG,1,01-01,...,14.990002,85.0,2,2,0.694336,0.479492,0.535645,0.5,0.713450,0.713450
4,a1000003,42,8.0,1,10.0,36.000000,0.0,PAYPALVC,0,01-01,...,0.000000,60.0,1,1,0.600586,0.606934,0.526367,0.0,0.573034,0.573034


In [3]:
df.customerID_prob[df.customerID_prob>1]=1

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [4]:
# Metode

In [5]:
#Konversi kategori/object ke angka (ordinal)
print "Konversi kategori/object ke angka (ordinal):"

#Cari kolom yang tipenya object, bukan integer maupun float
object_columns=df.loc[:, df.dtypes == object].columns


for col in object_columns:
    print col
    le=LabelEncoder()
    # Konversi deh
    df[col]=le.fit_transform(df[col])
    
if 'orderID' in df.columns:
    del df['orderID']

Konversi kategori/object ke angka (ordinal):
orderID
sizeCode
paymentMethod
mmdd


In [6]:
# Turunin presisi, iseng-iseng aja
float_64_columns = df.loc[:, df.dtypes == np.float64].columns
print "float_64_columns", float_64_columns.tolist()
for col in float_64_columns:
    df[col] = df[col].astype(np.float16)

int_64_columns = df.loc[:, df.dtypes == np.int64].columns
for col in int_64_columns:
    df[col] = df[col].astype(np.int32)
    

# Impute
df.rrp.fillna(df.rrp.median(), inplace=True)
df.productGroup.fillna(0, inplace=True)

print ''

Xdf=df.drop([label, 'productGroup'], axis=1)
print "Kolom yang dipakai: ", Xdf.columns.tolist()
X=Xdf.values
y=df.returnQuantity.values

Xdf_train, Xdf_test, ydf_train, ydf_test = train_test_split(Xdf, df.returnQuantity, test_size=0.15, random_state=42)

# Cara cek Nan
#Xdf.columns-Xdf.dropna(axis=1).columns

float_64_columns ['productGroup', 'price', 'rrp', 'voucherAmount', 'order_total', 'customer_budget', 'expense_ratio', 'unit_price', 'usual_unit_price', 'price_diff', 'after_voucher', 'articleID_prob', 'colorCode_prob', 'sizeCode_prob', 'customerID_prob', 'ac_prob', 'as_prob']

Kolom yang dipakai:  ['sizeCode', 'quantity', 'price', 'rrp', 'voucherAmount', 'paymentMethod', 'mmdd', 'months', 'order_total', 'customer_budget', 'expense_ratio', 'unit_price', 'usual_unit_price', 'price_diff', 'after_voucher', 'order_order', 'choice_order', 'articleID_prob', 'colorCode_prob', 'sizeCode_prob', 'customerID_prob', 'ac_prob', 'as_prob']


In [7]:
X_train, X_test, y_train, y_test = Xdf_train.values, Xdf_test.values, ydf_train.values, ydf_test.values

In [8]:
lr=LinearRegression(n_jobs=-1, normalize=True)
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
#y_pred=np.array([round(yp+0.00049) for yp in y_pred], dtype=np.int8)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[117083  50579     10      0      0      0]
 [ 31544 148974      6      0      0      0]
 [     8    533      4      0      0      0]
 [     0     19      7      0      0      0]
 [     0      2      1      0      0      0]
 [     0      0      5      0      0      0]]
82766


In [ ]:
#y_pred
df3=pd.DataFrame([y_pred, Xdf_test.quantity]); df3

# 82766
Kiri: True, Atas Predicted
```
[[117083  50579     10      0      0      0]
 [ 31544 148974      6      0      0      0]
 [     8    533      4      0      0      0]
 [     0     19      7      0      0      0]
 [     0      2      1      0      0      0]
 [     0      0      5      0      0      0]]
```

### Ini Importances?

In [45]:
from sklearn.feature_selection import RFE
lr=LinearRegression(n_jobs=-1, normalize=True)
selector = RFE(lr, 15, step=1)
selector = selector.fit(X, y)
pair=zip(Xdf.columns, selector.ranking_)
t1=[i for i in pair]; t1
sorted(t1, key=itemgetter(1))

[('quantity', 1),
 ('voucherAmount', 1),
 ('order_total', 1),
 ('expense_ratio', 1),
 ('unit_price', 1),
 ('usual_unit_price', 1),
 ('price_diff', 1),
 ('after_voucher', 1),
 ('choice_order', 1),
 ('articleID_prob', 1),
 ('colorCode_prob', 1),
 ('sizeCode_prob', 1),
 ('customerID_prob', 1),
 ('ac_prob', 1),
 ('as_prob', 1),
 ('paymentMethod', 2),
 ('sizeCode', 3),
 ('price', 4),
 ('customer_budget', 5),
 ('rrp', 6),
 ('order_order', 7),
 ('months', 8),
 ('mmdd', 9)]

In [8]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='sag', n_jobs=-1)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)
print confusion_matrix(y_test, y_pred_logreg)
print dmc_error(y_test, y_pred_logreg)

C:\Anaconda2\lib\site-packages\sklearn\linear_model\sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[[113961  53639     72      0      0      0]
 [ 33479 147009     36      0      0      0]
 [   130    385     30      0      0      0]
 [    10      7      9      0      0      0]
 [     0      0      3      0      0      0]
 [     0      0      5      0      0      0]]
88017


# 88017 Logistic Regression lama banget

```
[[113961  53639     72      0      0      0]
 [ 33479 147009     36      0      0      0]
 [   130    385     30      0      0      0]
 [    10      7      9      0      0      0]
 [     0      0      3      0      0      0]
 [     0      0      5      0      0      0]]
 ```

In [9]:
df.head()

,sizeCode,productGroup,quantity,price,rrp,voucherAmount,paymentMethod,returnQuantity,mmdd,months,...,price_diff,after_voucher,order_order,choice_order,articleID_prob,colorCode_prob,sizeCode_prob,customerID_prob,ac_prob,as_prob
0,16,3.0,1,10.0,29.984375,0.0,2,0,0,1,...,0.000000,30.0,1,1,0.421143,0.521484,0.516113,0.0,0.421143,0.421143
1,16,3.0,1,20.0,40.000000,0.0,2,0,0,1,...,5.000000,30.0,2,2,0.506836,0.492432,0.516113,0.0,0.475342,0.475342
2,13,8.0,1,35.0,50.000000,0.0,2,0,0,1,...,10.000000,85.0,1,1,0.642090,0.455322,0.535645,0.5,0.616699,0.616699
3,13,8.0,1,50.0,50.000000,0.0,2,1,0,1,...,14.992188,85.0,2,2,0.694336,0.479492,0.535645,0.5,0.713379,0.713379
4,15,8.0,1,10.0,36.000000,0.0,7,0,0,1,...,0.000000,60.0,1,1,0.600586,0.606934,0.526367,0.0,0.573242,0.573242


In [10]:
# rfr=RandomForestRegressor(20, n_jobs=3)
# rfr.fit(X_train, y_train)
# y_pred_rfr=rfr.predict(X_test)
# y_pred_rfr=np.array([round(yp) for yp in y_pred_rfr], dtype=np.int8)
# y_pred_rfr[y_pred_rfr<0]=0
# print confusion_matrix(y_test,y_pred_rfr)
# print dmc_error(y_test,y_pred_rfr)

In [11]:
# feature_importances(Xdf, rfr)

In [12]:
# y_pred_ens=(1*rfr.predict(X_test) + 9*model.predict(X_test))/10
# y_pred_ens=np.array([round(yp) for yp in y_pred_ens], dtype=np.int8)
# y_pred_ens[y_pred_ens<0]=0
# print confusion_matrix(y_test,y_pred_ens)
# print dmc_error(y_test,y_pred_ens)

In [ ]:
#lr=LinearRegression(n_jobs=-1, normalize=True)
#lr.fit(X_train, y_train**2)
#y_pred=lr.predict(X_test)
#y_pred=np.array([round(yp+0.00049) for yp in y_pred], dtype=np.int8)
#print y_pred
#y_pred=np.array([round(yp**0.5) for yp in y_pred])
#y_pred[y_pred<0]=0
#print confusion_matrix(y_test,y_pred)
#print dmc_error(y_test,y_pred)

```

```